Declaración de librerías e inicialización de valores

Ruta del dataset: https://www.tse.go.cr/zip/padron/padron_completo.zip 

In [1]:
import pandas as pd

import numpy as np
from datetime import datetime, timedelta
date = '31/12/2019'#init date
days=1
estado = {
    1: 'subceptible', 
    2: 'infectado',
    3: 'recuperado',
    4: 'fallecido'
}

Se lee los archivos csv(se hace de esta forma por que los caracteres especiales daban error)

In [4]:

with open('PADRON_COMPLETO.txt', 'r', encoding='utf8', errors='ignore') as pad:
    padron = pd.read_csv(pad, encoding='utf-8',header=None)

with open('Distelec.txt', 'r', encoding='utf8', errors='ignore') as dire:
    direcciones = pd.read_csv(dire, encoding='utf-8',header=None)


Se asigna un nombre a las columna

In [5]:
direcciones=direcciones.rename(columns={0: "CODELEC", 1: "PROVINCIA", 2: "CANTON", 3: "DISTRITO"})
padron=padron.rename(columns={0: "CEDULA", 1: "CODELEC", 2: "RELLENO", 3: "FECHACADUC", 4: "JUNTA", 5: "NOMBRE", 6: "PRIMER_APELLIDO", 7: "SEGUNDO_APPELIDO"})

Se hace merge de los dos fuentes para manejar un sólo dataset

In [6]:
padron_Completo=padron.merge(direcciones, left_on="CODELEC", right_on="CODELEC")
padron_Completo.head(2)

,CEDULA,CODELEC,RELLENO,FECHACADUC,JUNTA,NOMBRE,PRIMER_APELLIDO,SEGUNDO_APPELIDO,PROVINCIA,CANTON,DISTRITO
0,100339724,109007,1,20231119,0,JOSE,DELGADO,CORRALES,SAN JOSE,SANTA ANA,PIEDADES ...
1,101300341,109007,2,20220228,0,ERMIDA,SOLIS,FALLAS,SAN JOSE,SANTA ANA,PIEDADES ...


Se genera el dataset y se guardar en csv

In [16]:
#calcular días, se suma uno cada vez que se ejecuta este chunk
modified_date = datetime.strptime(date, '%d/%m/%Y') + timedelta(days=days)
actual_date=datetime.strftime(modified_date, '%d/%m/%Y')
days=days+1
#cantidad aleatorio de personas infectadas por día
cantidad_infectados = np.random.choice(range(1,999))
print('fecha: {0}, cantidad de infectados: {1}'.format(actual_date, cantidad_infectados))
#se saca la muestra con la cantidad de personas infectadas
infectados=padron_Completo.sample(n = cantidad_infectados)
#se le asigna un valor aleatorio que representa si está infectado...
infectados['ESTADO']=np.random.randint(1,5, size=cantidad_infectados)
#se cambian los valores por descripciones
infectados['ESTADO']= infectados['ESTADO'].apply(lambda x : estado[x])
#se sacan valores cercanos a 14 para determinar la cantidad de días contagiado
dias_infeccion = np.random.normal(14,1.0,1).astype(np.int)
#se asigna el día de contagia según el valor aleatorio
infectados['FECHA_CONTAGIO']=infectados['ESTADO'].apply(lambda x: datetime.strftime(datetime.strptime(actual_date, '%d/%m/%Y') - timedelta(days=np.random.normal(14,1.0,1).astype(np.int)[0].item()), '%d/%m/%Y') if (x!='subceptible' and  x!='infectado') else (actual_date if x=='infectado' else''))
#se asigna el día de recuperación
infectados['FECHA_RECUPERACION']=infectados['ESTADO'].apply(lambda x: actual_date if x=='recuperado' or x=='fallecido' else '')
#si determina el recontagio
infectados['RECONTAGIO']=infectados['ESTADO'].apply(lambda x: np.random.choice(['SI','NO']) if x=='recuperado' else '')
# se exporta a csv
#infectados.to_csv(('{0}.csv').format(actual_date.replace('/','')))
infectados


fecha: 07/01/2020, cantidad de infectados: 560


,CEDULA,CODELEC,RELLENO,FECHACADUC,JUNTA,NOMBRE,PRIMER_APELLIDO,SEGUNDO_APPELIDO,PROVINCIA,CANTON,DISTRITO,ESTADO,FECHA_CONTAGIO,FECHA_RECUPERACION,RECONTAGIO
612718,110790406,401001,1,20300204,0,ERICK MAURICIO,LOBO,HERNANDEZ,HEREDIA,CENTRAL,HEREDIA ...,recuperado,24/12/2019,07/01/2020,SI
1347960,107690773,109010,1,20240805,0,ROBERTO,ARIAS,CALVO,SAN JOSE,SANTA ANA,SAN RAFAEL ...,subceptible,,,
3030343,303110487,305070,2,20280207,0,YAMILETTE,DELGADO,GOMEZ,CARTAGO,TURRIALBA,CANADA ...,recuperado,25/12/2019,07/01/2020,SI
2373886,113910390,611005,1,20280420,0,IRVIN ANTONIO,SEVILLA,MORA,PUNTARENAS,GARABITO,HERRADURA ...,infectado,07/01/2020,,
471931,700470237,706006,2,20300313,0,EMILCE,GUILLEN,SOLIS,LIMON,GUACIMO,POCORA PARTE NORTE ...,fallecido,24/12/2019,07/01/2020,
